#Instalação Bibliotecas

In [ ]:
pip install pyspark

In [ ]:
pip install fsspec

In [ ]:
pip install gcsfs

#Bibliotecas

In [ ]:
#Importando funcções padrão. Função Window também importada porque será usado mais na frente
import os
from google.cloud import storage

from pyspark.sql import SparkSession, Window
spark = SparkSession.builder.appName('GCSFilesRead').getOrCreate()

import pyspark.sql.functions as F
from pyspark.sql.functions import to_timestamp, to_date

from pyspark.sql.types import *
import pandas as pd

In [ ]:
serviceaccount = 'gs://projetofinalgrupo8/chave/projetofinalgrupo8-2dcd866c3f46.json'
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = serviceaccount

dfspark = pd.read_csv('gs://projetofinalgrupo8/saida/data_tratado_pyspark.csv', sep=',',encoding='UTF-8', header=0)
df = spark.createDataFrame(dfspark).createOrReplaceTempView("leitura_sql")

#Consultas

**00.CONSULTA TABELA**

In [ ]:
spark.sql("SELECT * FROM leitura_sql LIMIT 10").show()

+-----------+--------------------+--------------------+--------------------+-----------------+--------+------------+----------------+-------------------------+----------------------+----+---------------+-----------------+-------------+-------------------------+----------------------------+---------------------------+
|   id_video|        titulo_video|            id_canal|          nome_canal|cont_visualizacao|curtidas|nao_curtidas|cont_comentarios|comentarios_desabilitados|curtidas_desabilitadas|pais|      categoria|publicado_em_data|data_destaque|like_por_visualizacao_100|deslike_por_visualizacao_100|coment_por_visualizacao_100|
+-----------+--------------------+--------------------+--------------------+-----------------+--------+------------+----------------+-------------------------+----------------------+----+---------------+-----------------+-------------+-------------------------+----------------------------+---------------------------+
|s9FH4rDMvds|LEVEI UM FORA? FI...|UCGfBwrCo

**01.CONSULTA VIDEOS COM MAIS DE 50% DE CURTIDAS EM RELAÇÃO AS VISUALIZAÇÕES**

In [ ]:
#spark.sql("SELECT * FROM leitura_sql WHERE pais = 'BR' AND like_por_visualizacao_100 > 50 AND cont_visualizacao > 0").show()
spark.sql("SELECT * FROM leitura_sql WHERE like_por_visualizacao_100 > 50 AND cont_visualizacao > 0").show()

+-----------+--------------------------+--------------------+--------------------+-----------------+--------+------------+----------------+-------------------------+----------------------+----+----------------+-----------------+-------------+-------------------------+----------------------------+---------------------------+
|   id_video|              titulo_video|            id_canal|          nome_canal|cont_visualizacao|curtidas|nao_curtidas|cont_comentarios|comentarios_desabilitados|curtidas_desabilitadas|pais|       categoria|publicado_em_data|data_destaque|like_por_visualizacao_100|deslike_por_visualizacao_100|coment_por_visualizacao_100|
+-----------+--------------------------+--------------------+--------------------+-----------------+--------+------------+----------------+-------------------------+----------------------+----+----------------+-----------------+-------------+-------------------------+----------------------------+---------------------------+
|CuyTC8FLICY|      Ani

**02.CONSULTA VIDEOS DA CATEGORIA FILMES E ANIMAÇÃO NOS EUA COM SUAS VISUALIZAÇÕES E PERCENTUAL DE CURTIDAS POR VISUALIZAÇÃO**

In [ ]:
spark.sql("SELECT titulo_video, MAX(cont_visualizacao) AS total_visualizacao_por_video, \
                  ROUND((MAX(curtidas) / MAX(cont_visualizacao))*100, 2) AS curtidas_cada_100_visualizacao \
           FROM leitura_sql \
           WHERE (categoria = 'filme_e_animacao' AND pais = 'US') \
           GROUP BY titulo_video \
           ORDER BY total_visualizacao_por_video DESC \
           LIMIT 10").show()

+--------------------+----------------------------+------------------------------+
|        titulo_video|total_visualizacao_por_video|curtidas_cada_100_visualizacao|
+--------------------+----------------------------+------------------------------+
|THE BATMAN – Main...|                    31990632|                          3.09|
|F9 - Official Tra...|                    23430400|                          0.94|
|Friday Night Funk...|                    21184728|                           2.5|
|Did you know that...|                    18390451|                          6.06|
|David Blaine Asce...|                    17212348|                          2.47|
|Friends: The Reun...|                    16143739|                           2.2|
|Parkour - Animati...|                    16031212|                          3.13|
|THE CONJURING: TH...|                    15396299|                          1.53|
|Wonder Woman 1984...|                    14810365|                          1.86|
|Pea

**03.CONSULTA VÍDEOS QUE TENHAM O NOME DO BTS COM MAIS VISUALIZAÇÕES**

In [ ]:
spark.sql("SELECT titulo_video, MAX(cont_visualizacao) AS visualizacoes \
           FROM leitura_sql \
           WHERE (titulo_video LIKE '%BTS%') \
           GROUP BY titulo_video \
           ORDER BY visualizacoes DESC \
           LIMIT 5").show()

+-------------------------+-------------+
|             titulo_video|visualizacoes|
+-------------------------+-------------+
|BTS (방탄소년단) 'Butt...|    296314174|
|BTS (방탄소년단) 'Dyna...|    262319276|
|BTS (방탄소년단) 'Perm...|    194795844|
|BTS (방탄소년단) 'Life...|    161912058|
|BTS (방탄소년단) 'Film...|     78893765|
+-------------------------+-------------+



**04.CONSULTA VÍDEOS QUE TENHAM O NOME DO NEYMAR COM MAIS VISUALIZAÇÕES**

In [ ]:
spark.sql("SELECT titulo_video, MAX(cont_visualizacao) AS visualizacoes \
           FROM leitura_sql \
           WHERE (titulo_video LIKE '%NEYMAR%') \
           GROUP BY titulo_video \
           ORDER BY visualizacoes DESC \
           LIMIT 10").show()

+--------------------+-------------+
|        titulo_video|visualizacoes|
+--------------------+-------------+
|SHOW DE NEYMAR! B...|      5761326|
|NEYMAR MARCA DUAS...|      4464807|
|PAQUETÁ MARCA, MA...|      3878196|
|♫ NEYMAR TÁ ON!!!...|      3847530|
|NEYMAR DÁ ASSISTÊ...|      2372773|
|NEYMAR JOGA MUITO...|      2244898|
|♫ VAI TER NEYMAR ...|      2168694|
|♫ DEU PSG!!! NEYM...|      2122704|
|A CHEGADA SENSACI...|      2078345|
|New *NEYMAR JR* U...|      1942078|
+--------------------+-------------+



**05.CONSULTA DE VIDEOS COM MAIS DE 10 MILHÕES DE VISUALIZAÇÕES DA CATEGORIA ESPORTES DO YOUTUBE BRASIL**

In [ ]:
spark.sql("SELECT titulo_video, MAX(cont_visualizacao) AS visualizacoes \
           FROM leitura_sql \
           WHERE (cont_visualizacao > 10000000 AND categoria = 'esportes' AND pais = 'BR') \
           GROUP BY titulo_video \
           ORDER BY visualizacoes DESC \
           LIMIT 10").show()

+--------------------+-------------+
|        titulo_video|visualizacoes|
+--------------------+-------------+
|The Weeknd’s FULL...|     27865785|
|#LEOMESSI: First ...|     26294018|
|🎥  LEO MESSI,  D...|     20648273|
|Khabib Nurmagomed...|     19819443|
|Amazing moment of...|     10326487|
+--------------------+-------------+



**06.CONSULTA VIDEOS ANITTA COM NUMERO DE VISUALIZACAO, CURTIDAS E COMENTARIOS PUBLICADOS DEPOIS DE 2021-06-30** 

In [ ]:
spark.sql("SELECT titulo_video, MAX(cont_visualizacao) AS num_visualizacao, MAX(curtidas) AS num_curtidas, MAX(cont_comentarios) AS num_comentarios \
           FROM leitura_sql \
           WHERE nome_canal = 'Anitta' AND publicado_em_data > '2021-06-30' \
           GROUP BY titulo_video \
           ORDER BY num_visualizacao DESC \
           LIMIT 10").show()

+--------------------+----------------+------------+---------------+
|        titulo_video|num_visualizacao|num_curtidas|num_comentarios|
+--------------------+----------------+------------+---------------+
|Anitta - Faking L...|         6890459|      347663|          45614|
|Anitta - Envolver...|         5087138|      363785|          36343|
|Anitta - Faking L...|          423824|       85881|          14683|
|Anitta Sings Girl...|          202367|       32648|           2658|
+--------------------+----------------+------------+---------------+



**07.CONSULTA VIDEOS ANITTA COM NUMERO DE VISUALIZACAO, CURTIDAS E COMENTARIOS**

In [ ]:
spark.sql("SELECT titulo_video, MAX(cont_visualizacao) AS num_visualizacao, MAX(curtidas) AS num_curtidas, MAX(cont_comentarios) AS num_comentarios \
           FROM leitura_sql \
           WHERE nome_canal = 'Anitta' \
           GROUP BY titulo_video \
           ORDER BY num_visualizacao DESC \
           LIMIT 10").show()

+--------------------+----------------+------------+---------------+
|        titulo_video|num_visualizacao|num_curtidas|num_comentarios|
+--------------------+----------------+------------+---------------+
|Anitta Me Gusta (...|        26165923|     1257868|         348858|
|Anitta - Girl Fro...|        15771509|     1035748|         150062|
|Anitta - Loco (Of...|         7982621|      310779|          45966|
|Anitta - Faking L...|         6890459|      347663|          45614|
|Anitta - Girl Fro...|         5264455|      156690|          10821|
|Anitta - Envolver...|         5087138|      363785|          36343|
|Anitta Me Gusta (...|         1448503|      149877|          11669|
|Anitta Performanc...|         1373409|      113345|           9718|
|Anitta | RELEASED...|          989410|       41066|           1165|
|Anitta on RELEASE...|          945532|       20102|           2352|
+--------------------+----------------+------------+---------------+



**08.CONSULTA DE TOTAL DE VISUALIZACOES E NUMERO DE VIDEOS DOS CANAIS DA CATEGORIA DE JOGOS**

In [ ]:
spark.sql("SELECT nome_canal, COUNT(DISTINCT titulo_video) AS numero_videos, SUM(DISTINCT cont_visualizacao) AS total_visualizacao_por_canal \
           FROM leitura_sql \
           WHERE categoria = 'jogos' \
           GROUP BY nome_canal \
           ORDER BY total_visualizacao_por_canal DESC \
           LIMIT 10").show()

+--------------------+-------------+----------------------------+
|          nome_canal|numero_videos|total_visualizacao_por_canal|
+--------------------+-------------+----------------------------+
|      MrBeast Gaming|           61|                  4258737435|
|         Brawl Stars|           30|                  2667075201|
|             SSundee|          100|                  2090914442|
|      Clash of Clans|           40|                  1906048141|
|               Dream|           11|                  1561489175|
|       AnthonySenpai|           11|                  1262361580|
|Free Fire India O...|           48|                  1249355714|
|         Klem Family|            7|                  1228945428|
|   League of Legends|           36|                  1185810637|
|          TommyInnit|           32|                  1091651516|
+--------------------+-------------+----------------------------+



**09.CONSULTA NUMERO DE CANAIS E MEDIA DE VISUALIZACOES POR VIDEO DA CATEGORIA**

In [ ]:
spark.sql("SELECT categoria, COUNT(DISTINCT nome_canal) AS numero_canais, ROUND(MEAN(cont_visualizacao), 2) AS media_visualizacao_por_video \
           FROM leitura_sql \
           GROUP BY categoria \
           ORDER BY media_visualizacao_por_video DESC").show()

+--------------------+-------------+----------------------------+
|           categoria|numero_canais|media_visualizacao_por_video|
+--------------------+-------------+----------------------------+
|              musica|         5786|                  4239143.85|
|ciencias_e_tecnol...|          784|                  2108160.57|
|               jogos|         3469|                  1749334.19|
|      entretenimento|         7338|                  1710892.29|
|             comedia|         1503|                  1571663.04|
|     pessoas_e_blogs|         5134|                  1359910.64|
|    filme_e_animacao|         1301|                  1308026.47|
|            educacao|          735|                  1266638.11|
|            esportes|         2008|                  1187144.65|
|  como_faz_e_estilos|         1292|                   878036.94|
|              outros|          137|                   835704.84|
|             animais|          286|                    793950.2|
|noticias_

**10.CONSULTA VIDEOS COM MENOR NUMERO DE DESCURTIDAS MAIORES QUE ZERO NA CATEGORIA MUSICA NOS EUA, MOSTRANDO NÚMERO DE VISUALIZAÇÕES**

In [ ]:
spark.sql("SELECT titulo_video, MIN(nao_curtidas) AS menores_nao_curtidas, MAX(cont_visualizacao) AS numero_visualizacao \
           FROM leitura_sql \
           WHERE (categoria = 'musica' AND pais = 'US' AND nao_curtidas > 0) \
           GROUP BY titulo_video \
           ORDER BY menores_nao_curtidas \
           LIMIT 10").show()

+--------------------+--------------------+-------------------+
|        titulo_video|menores_nao_curtidas|numero_visualizacao|
+--------------------+--------------------+-------------------+
|Opry Livestream -...|                   7|              56447|
|Morgan Wallen - S...|                   9|             580319|
|      Beautiful Trip|                  24|             679656|
|Sleepy Hallow - 1...|                  28|             525397|
|Carole King - So ...|                  30|             527981|
|Phish Dinner And ...|                  34|             127299|
|Sammy Hagar Remem...|                  36|             351147|
|Eddie Van Halen's...|                  37|             595557|
|Sammy Hagar Made ...|                  37|             207672|
|Morgan Wallen - D...|                  39|             259746|
+--------------------+--------------------+-------------------+



**11.CONSULTA VIDEOS DO CANAL DO FELIPE NETO COM MAIOR NÚMERO DE VISUALIZAÇÕES**

In [ ]:
spark.sql("SELECT titulo_video, MAX(cont_visualizacao) AS numero_visualizacao \
           FROM leitura_sql \
           WHERE nome_canal = 'Felipe Neto' \
           GROUP BY titulo_video \
           ORDER BY numero_visualizacao DESC \
           LIMIT 10").show()

+--------------------+-------------------+
|        titulo_video|numero_visualizacao|
+--------------------+-------------------+
|SHOW DA BLACK FRI...|            5533110|
|AMONG US COM NETO...|            5178230|
|AMONG US / NETOLA...|            4894976|
|LIVE / AMONG US c...|            4292845|
|AMONG US COM NAVA...|            3943216|
|ENTREI NO ROUND 6!!!|            3922688|
|LIVE - AMONG US -...|            3675967|
|NOVA BATALHA! SIR...|            3465543|
|LIVE DE 40 MILHÕE...|            3192567|
|FIZ O PIOR MILKSH...|            3148609|
+--------------------+-------------------+



**12.CONSULTA CANAIS BRASILEIROS COM MAIOR NÚMERO DE VÍDEOS PUBLICADOS**

In [ ]:
spark.sql("SELECT nome_canal, COUNT(DISTINCT titulo_video) AS numero_videos_destaque \
           FROM leitura_sql \
           WHERE pais = 'BR' \
           GROUP BY nome_canal \
           ORDER BY numero_videos_destaque DESC \
           LIMIT 10").show()

+------------------+----------------------+
|        nome_canal|numero_videos_destaque|
+------------------+----------------------+
|                ge|                   322|
|         A Fazenda|                   207|
|       Felipe Neto|                   166|
|          De Placa|                   142|
| MasterChef Brasil|                    94|
|   WebTVBrasileira|                    86|
|       FutParódias|                    86|
|   Invento na Hora|                    76|
|Free Fire - Brasil|                    69|
|            AM3NlC|                    66|
+------------------+----------------------+



**13.CONSULTA NÚMERO DE CANAIS POR PAÍS E SUA MÉDIA DE COMENTÁRIOS POR VÍDEO**

In [ ]:
spark.sql("SELECT pais, COUNT(DISTINCT nome_canal) AS numero_canais, ROUND(MEAN(cont_comentarios), 2) AS media_comentarios_por_video \
           FROM leitura_sql \
           GROUP BY pais \
           ORDER BY numero_canais DESC").show()

+----+-------------+---------------------------+
|pais|numero_canais|media_comentarios_por_video|
+----+-------------+---------------------------+
|  RU|         5345|                    5501.33|
|  DE|         5015|                    8761.68|
|  CA|         4923|                   13011.36|
|  US|         4547|                   13990.23|
|  GB|         4398|                   11287.87|
|  FR|         3636|                    7439.54|
|  BR|         3317|                    10544.9|
|  MX|         3078|                   13719.64|
|  IN|         2799|                   13110.12|
|  JP|         2714|                    7991.05|
|  KR|         2600|                   10752.21|
+----+-------------+---------------------------+



**14.CONSULTA VIDEOS COM MAIS VISUALIZAÇÕES**

In [ ]:
spark.sql("SELECT titulo_video, MAX(cont_visualizacao) AS total_visualizacao \
           FROM leitura_sql \
           GROUP BY titulo_video \
           ORDER BY total_visualizacao DESC").show()

+-------------------------+------------------+
|             titulo_video|total_visualizacao|
+-------------------------+------------------+
|BTS (방탄소년단) 'Butt...|         296314174|
|BTS (방탄소년단) 'Dyna...|         262319276|
|     Turn into orbeez ...|         206202284|
|     Filhaal2 Mohabbat...|         202091414|
|BTS (방탄소년단) 'Perm...|         194795844|
|     JETSKI WAX PRANK!...|         194625542|
|      LISA - 'LALISA' M/V|         192376395|
|     BLACKPINK - 'Ice ...|         184778248|
|     INSANE strawberry...|         164362471|
|BTS (방탄소년단) 'Life...|         161912058|
|     BLACKPINK – ‘Love...|         161416953|
|     KGF Chapter2 TEAS...|         156704924|
|     Beach Money Ball!...|         153462028|
|     Paytm IPL 2021 Ad...|         141191928|
|     Adele - Easy On M...|         139547582|
|     When the cheating...|         137308926|
|     She is foxy but n...|         131108950|
|     Guru Randhawa: Na...|         130124523|
|     MvRyhan Funny vid...|      